In [1]:
%pwd

'c:\\Users\\Kshitij\\Downloads\\DPDZero\\DPDZero\\research'

In [2]:
import os

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Kshitij\\Downloads\\DPDZero\\DPDZero'

In [5]:
import pandas as pd

In [9]:
call_logs = pd.read_csv('artifacts/data_ingestion/call_logs.csv')
agent_roster = pd.read_csv('artifacts/data_ingestion/agent_roster.csv')
disposition_summary = pd.read_csv('artifacts/data_ingestion/disposition_summary.csv')

In [17]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataMergingConfig:
    root_dir: Path
    data_dir: Path
    merged_data_dir: Path

In [18]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml,create_directories

In [19]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_merging_config(self) -> DataMergingConfig:
        config = self.config.data_merging

        create_directories([config.root_dir])

        data_merging_config = DataMergingConfig(
            root_dir = config.root_dir,
            data_dir = config.data_dir,
            merged_data_dir = config.merged_data_dir
        )
        return data_merging_config

In [20]:
import pandas as pd
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [26]:
class DataMerging:
    def __init__(self, config: DataMergingConfig):
        self.config = config

    def merge_data(self):
        call_logs = pd.read_csv(self.config.data_dir + '/call_logs.csv')
        agent_roster = pd.read_csv(self.config.data_dir + '/agent_roster.csv')
        disposition_summary = pd.read_csv(self.config.data_dir + '/disposition_summary.csv')

        call_logs['call_date'] = pd.to_datetime(call_logs['call_date'])
        disposition_summary['call_date'] = pd.to_datetime(disposition_summary['call_date'])

        agent_login = pd.merge(
            agent_roster,
            disposition_summary,
            on=['agent_id', 'org_id'],
            how='left'
        )

        merged_df = pd.merge(
            agent_login,
            call_logs,
            on=['agent_id', 'org_id', 'call_date'],
            how='left'
        )
        merged_df.to_csv(self.config.merged_data_dir, index=False)
        
        logger.info(f"Data merged and saved to {self.config.merged_data_dir}")
        

In [27]:
try:
    config = ConfigurationManager()
    data_merging_config = config.get_data_merging_config()
    data_merging = DataMerging(config=data_merging_config)
    data_merging.merge_data()
except Exception as e:
    raise e


[2025-05-03 19:16:08,190: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-03 19:16:08,196: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-03 19:16:08,200: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-03 19:16:08,204: INFO: common: created directory at: artifacts]
[2025-05-03 19:16:08,207: INFO: common: created directory at: artifacts/data_merging]
[2025-05-03 19:16:08,254: INFO: 3098861574: Data merged and saved to artifacts/data_merging/merged_data.csv]
